## Installing and importing dependencies

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

In [ ]:
import mediapipe as mp
import cv2

In [ ]:
mp_draw = mp.solutions.drawing_utils
mp_model = mp.solutions.holistic    #model

In [ ]:
#if you want to customize the results detected by the model
#first landmark argument then connection argument in draw_landmarks function as above form

mp_draw.DrawingSpec(color=(0,0,255),thickness=2,circle_radius=2)

## Extracting result format and testing our initial setup

In [ ]:
cap = cv2.VideoCapture('./test-01.mp4')

with mp_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    ret, frame = cap.read()
    while ret:    #cap.isOpened():
        
        #ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        image.flags.writeable = False
        results = holistic.process(image)
        #print(results.face_landmarks)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #face
        mp_draw.draw_landmarks(image, results.face_landmarks, mp_model.FACEMESH_TESSELATION,
                               mp_draw.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                               mp_draw.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        #pose
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_model.POSE_CONNECTIONS)
        
        cv2.imshow('Output',image)
        cv2.waitKey(1)
        
        ret, frame = cap.read()
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
'''if cv2.waitKey(25) and 0xff == ord('q'):
       break'''

## Initializing our training dataset. Which is a CSV file

In [ ]:
import csv
import os
import numpy as np

In [ ]:
num_cords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
landmarks = ['class']

for i in range(1,num_cords+1):
    landmarks += [f'x{str(i)}',f'y{str(i)}',f'z{str(i)}',f'v{str(i)}']

In [ ]:
with open('training-cords.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## Adding training data

In [ ]:
cap = cv2.VideoCapture('./Dataset/Train/angry-01.mp4')
class_name = 'Angry'

with mp_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    ret, frame = cap.read()
    while ret:    #cap.isOpened():
        
        #ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        image.flags.writeable = False
        results = holistic.process(image)
        #print(results.face_landmarks)
        image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #face
        mp_draw.draw_landmarks(image, results.face_landmarks, mp_model.FACEMESH_TESSELATION,
                               mp_draw.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                               mp_draw.DrawingSpec(color=(255,255,255),thickness=1,circle_radius=1))
        
        #pose
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_model.POSE_CONNECTIONS)
        
        try:
            pose = results.pose_landmarks.landmark
            pose_row = np.array([[lm.x,lm.y,lm.z,lm.visibility] for lm in pose]).flatten()

            face = results.face_landmarks.landmark
            face_row = np.array([[lm.x,lm.y,lm.z,lm.visibility] for lm in face]).flatten()

            row = list(pose_row) + list(face_row)
            row.insert(0,class_name)

            with open('training-cords.csv',mode='a',newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
        except:
            pass

        
        cv2.imshow('Output',image)
        cv2.waitKey(1)
        
        ret, frame = cap.read()

        '''if cv2.waitKey(25) and 0xff == ord('q'):
            break'''

cap.release()
cv2.destroyAllWindows()